In [ ]:
# LSTM for shipment forecasting problem with regression framing
import numpy
import matplotlib.pyplot as plt
from pandas import read_csv
from pandas import DataFrame
from pandas import Series
import math
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import LSTM, GRU, LeakyReLU
from sklearn.preprocessing import MinMaxScaler, StandardScaler, MaxAbsScaler, RobustScaler, Normalizer
from sklearn.metrics import mean_squared_error
import keras.optimizers as optimize
from keras.callbacks import CSVLogger, ModelCheckpoint
from keras.regularizers import L1L2
ADAM = optimize.Adam(lr=0.0002, beta_1=0.9, beta_2=.999, decay=0.00004)

# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back):  #for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        #print(len(dataX))
        dataY.append(dataset[i + look_back, 0])
        #print(len(dataY))
    return numpy.array(dataX), numpy.array(dataY)

# fix random seed for reproducibility
numpy.random.seed(7)

# load the dataset
dataframe = read_csv('../datasets/LSTM_PL29_Rev2.csv', usecols=[1], engine='python', skipfooter=0)
dataset = dataframe.values
dataset = dataset.astype('float32')

#RobustScaler the dataset
scaler = RobustScaler(with_centering=True, with_scaling=True, quantile_range=(25.0, 75.0), copy=True)
dataset = scaler.fit_transform(dataset)
dataset_df = DataFrame(dataset)

# split into train and test sets
train_size = int(len(dataset) * .77)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size+4,:], dataset[train_size+1:len(dataset)+1,:]
print(len(dataset))
print(train_size)
print(len(train))
print(test_size)
print(len(test))

# reshape into X=t and Y=t+1
look_back = 3
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

# reshape input to be [samples, time steps, features]
trainX = numpy.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = numpy.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

# create and fit the LSTM network
model = Sequential()

model.add(LSTM(300, activation='tanh', recurrent_activation='hard_sigmoid', 
               kernel_regularizer=L1L2(l1=0.0,l2=0.0), 
               recurrent_regularizer=L1L2(l1=0.0,l2=0.0), 
               bias_regularizer=L1L2(l1=0.0,l2=0.0), 
               activity_regularizer=L1L2(l1=0.0,l2=0.0), 
               dropout=0.0, recurrent_dropout=0.0, 
               return_sequences=True, stateful=False, 
               use_bias=True, input_shape=(1, look_back)))

model.add(LSTM(200, activation='tanh', recurrent_activation='hard_sigmoid', 
               kernel_regularizer=L1L2(l1=0.0,l2=0.0), 
               recurrent_regularizer=L1L2(l1=0.0,l2=0.0), 
               bias_regularizer=L1L2(l1=0.0,l2=0.0), 
               activity_regularizer=L1L2(l1=0.00,l2=0.0), 
               dropout=0.0, recurrent_dropout=0.0, 
               return_sequences=True, stateful=False))

model.add(LSTM(100, activation='tanh', recurrent_activation='sigmoid',
               return_sequences=True, stateful=False, 
               kernel_regularizer=L1L2(l1=0.0,l2=0.0), 
               recurrent_regularizer=L1L2(l1=0.0,l2=0.0), 
               bias_regularizer=L1L2(l1=0.0,l2=0.0), 
               activity_regularizer=L1L2(l1=0.0,l2=0.0), 
               dropout=0.0))

model.add(LSTM(50, activation='tanh', recurrent_activation='tanh',
               return_sequences=False, stateful=False, 
               kernel_regularizer=L1L2(l1=0.0,l2=0.0), 
               recurrent_regularizer=L1L2(l1=0.0,l2=0.0), 
               bias_regularizer=L1L2(l1=0.0,l2=0.0), 
               activity_regularizer=L1L2(l1=0.0,l2=0.0), 
               dropout=0.0))

model.add(Dense(1, activation= 'linear'))

model.compile(loss='mean_squared_error', optimizer=ADAM, metrics= ['mse'])

history = model.fit(trainX, trainY, epochs=2500, batch_size=1, verbose=0, validation_data=(testX, testY), shuffle=False) 

#Saved Trained Model
model.save("../datasets/LSTM_PL29_trained_model_Take4.h5")

# plot history
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show()

# make predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

# invert predictions
trainPredict = scaler.inverse_transform(trainPredict)

trainPredict_df = DataFrame(trainPredict)
trainPredict_df.to_csv('../datasets/LSTM_trainPredict_dataset.csv')

trainY = scaler.inverse_transform([trainY])

testPredict = scaler.inverse_transform(testPredict)

testPredict_df = DataFrame(testPredict)
testPredict_df.to_csv('../datasets/LSTM_testPredict_dataset.csv')

testY = scaler.inverse_transform([testY])

# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainY[0], trainPredict[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

# shift train predictions for plotting
trainPredictPlot = numpy.empty_like(dataset)
trainPredictPlot[:, :] = numpy.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict

# shift test predictions for plotting
testPredictPlot = numpy.empty_like(dataset)
testPredictPlot[:, :] = numpy.nan
testPredictPlot[len(trainPredict)+(3):len(dataset)+1, :] = testPredict
print(len(dataset))
print(len(testPredict))

# plot baseline and predictions
plt.plot(scaler.inverse_transform(dataset))
plt.plot(trainPredictPlot)
plt.plot(testPredictPlot)
plt.savefig('../datasets/forecast8.png')
plt.show()

Using TensorFlow backend.


39
30
34
9
8
